In [1]:
'''
This code opens the Microsoft Academic Graph Dataset and trains HGT
Based on code provided by original HGT paper
'''
import torch
from hgt import *
from hgt_utils import *
from model import *
from ogb.nodeproppred import PygNodePropPredDataset
from ogb.nodeproppred import Evaluator
import multiprocessing as mp
import argparse
import numpy as np
import time
import pandas as pd
import matplotlib as plt
import seaborn as sb

print("Microsoft Academic Graph Dataset Experiment")

Microsoft Academic Graph Dataset Experiment


In [2]:
'''
Data Preprocessing
ogbn-mag only comes with paper node features, thus for other nodes types we take the average
of connected paper nodes as input features. 
'''
print("Begin Data Preprocessing")
print("")
print("Retrieving Data from Open Graph Benchmark ...")

# Get dataset using Pytorch Geometric Loader
dataset = PygNodePropPredDataset(name='ogbn-mag')
print("... Retrieval complete")
data = dataset[0] # pyg graph object

Begin Data Preprocessing

Retrieving Data from Open Graph Benchmark ...
... Retrieval complete


In [3]:
evaluator = Evaluator(name='ogbn-mag')

# Preparing Graph
graph, y, train_paper, valid_paper, test_paper = prepare_graph(data, dataset)

Populating edge lists into Graph object
('author', 'affiliated_with', 'institution')
('author', 'writes', 'paper')
('paper', 'cites', 'paper')
('paper', 'has_topic', 'field_of_study')

Reformatting edge lists and computing node degrees
institution author affiliated_with 8740
author institution rev_affiliated_with 852987
author paper rev_writes 1134649
paper author writes 736389
paper paper cites 629169
paper paper rev_cites 617924
paper field_of_study rev_has_topic 736389
field_of_study paper has_topic 59965

Constructing node feature vectors for each node type in graph
author
field_of_study
institution
paper

Constructing Node features for institutions

Splitting dataset into train, val and test

Creating Masks

Preprocessing complete


In [4]:
'''
Creating Model
'''
print("Creating Model")
hgt_GNN = HGTModel(len(graph.node_feature['paper'][0]), # input_dim
                   256,                                 # hidden_dim
                   len(graph.get_types()),              # num_node_types
                   len(graph.get_meta_graph()),         # num_edge_types
                   8,                                   # num_heads
                   4,                                   # num_layers
                   0.2,                                 # dropout
                   prev_norm = True,                    # normalization on all but last layer
                   last_norm = False,                   # normalization on last layer
                   use_rte = False)                     # use relative temporal encoding 
classifier = Classifier(256, graph.y.max()+1)

print(f'Classifier output dim is: {graph.y.max()+1}')

HGT_classifier = nn.Sequential(hgt_GNN, classifier)

print(HGT_classifier)

Creating Model
Classifier output dim is: 349
Sequential(
  (0): HGTModel(
    (adapt_features): ModuleList(
      (0-3): 4 x Linear(in_features=129, out_features=256, bias=True)
    )
    (hgt_layers): ModuleList(
      (0-3): 4 x HGTLayer()
    )
    (drop): Dropout(p=0.2, inplace=False)
  )
  (1): Classifier(n_hid=256, n_out=349)
)


In [ ]:
'''
Preprocessing data

'''
batch_number = 32 # number of sampled graphs for each epoch
batch_size = 128
num_epochs = 10
num_workers = 8
clip = 1.0
sample_depth = 6
sample_width = 520
plot = False # True or false to plot data

save_directory = r"C:\Users\johns\OneDrive\Desktop\HGT_Data\models"
# device = torch.device() DEVICE CONTROL POSSIBLE
target_nodes = np.arange(len(graph.node_feature['paper']))

# Negative Log Likelihood Loss
criterion = nn.NLLLoss()

# Get list of model parameters w/ associated names
parameters_optimizer = list(HGT_classifier.named_parameters())
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in parameters_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in parameters_optimizer if any(nd in n for nd in no_decay)],     'weight_decay': 0.0}
]

# AdamW optimizer w/specified parameter groups and epsilon value
optimizer = torch.optim.AdamW(optimizer_grouped_parameters, eps=1e-06)

# Create a OneCycleLR learning rate scheduler
scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, pct_start=0.05, anneal_strategy='linear', final_div_factor=10,\
                        max_lr = 5e-4, total_steps = batch_size * num_epochs + 1)

stats = []
result = []
best_val = 0
training_step = 0

def randint():
    return np.random.randint(2**31 - 1)

def ogbn_sample(seed, samp_nodes):
    np.random.seed(seed)
    ylabel      = torch.LongTensor(graph.y[samp_nodes])
    feature, times, edge_list, indxs, _ = sample_subgraph(graph, \
                inp = {'paper': np.concatenate([samp_nodes, graph.years[samp_nodes]]).reshape(2, -1).transpose()}, \
                sampled_depth = sample_depth, sampled_number = sample_width, \
                    feature_extractor = feature_MAG)
    node_feature, node_type, edge_time, edge_index, edge_type, node_dict, edge_dict = \
            to_torch(feature, times, edge_list, graph)
    train_mask = graph.train_mask[indxs['paper']]
    valid_mask = graph.valid_mask[indxs['paper']]
    test_mask  = graph.test_mask[indxs['paper']]
    ylabel     = graph.y[indxs['paper']]
    return node_feature, node_type, edge_time, edge_index, edge_type, (train_mask, valid_mask, test_mask), ylabel

def prepare_data(pool, task_type = 'train', s_idx = 0, n_batch = batch_number, batch_size = batch_size):
    '''
        Sampled and prepare training and validation data using multi-process parallization.
    '''
    jobs = []
    if task_type == 'train':
        for batch_id in np.arange(n_batch):
            print(f'starting preprocessing batch: {batch_id}')
            p = pool.apply_async(ogbn_sample, args=([randint(), \
                            np.random.choice(target_nodes, batch_size, replace = False)]))
            jobs.append(p)
            print(f'finished preprocessing batch: {batch_id}')
    elif task_type == 'sequential':
        for i in np.arange(n_batch):
            target_papers = graph.test_paper[(s_idx + i) * batch_size : (s_idx + i + 1) * batch_size]
            p = pool.apply_async(ogbn_sample, args=([randint(), target_papers]))
            jobs.append(p)
    elif task_type == 'variance_reduce':
        target_papers = graph.test_paper[s_idx * batch_size : (s_idx + 1) * batch_size]
        for batch_id in np.arange(n_batch):
            p = pool.apply_async(ogbn_sample, args=([randint(), target_papers]))
            jobs.append(p)
    print("Preprocessing complete ---------------------------------------------------")
    return jobs

pool = mp.Pool(num_workers)

# start time
start_time = time.time()
jobs = prepare_data(pool)

# Begin Training
print("Beginning Training")
print("")
train_step = 0
epoch_tracker = 1
print("starting loop")

print(f'length of jobs is: {len(jobs)}')
print(f'type of jobs is: {type}')

